In [1]:
import findspark
findspark.init()

In [2]:
import pandas as pd
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date,datetime,timedelta


In [6]:
sc = SparkSession.builder.appName("PysparkExample").config("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g").config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()

<b>Read a dataset</b>

In [8]:
dataframe=sc.read.json("./nyt2.json")
dataframe.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|       Little, Br

<b><h2>Duplicate Values</h2></b>

<p>Duplicate values in a table can be eliminated by using dropDuplicates() function.</p>

In [9]:
data_drop=dataframe.drop_duplicates()
data_drop.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+----------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|{{1213401600000}}|Juan Cabrillo and...|{26.95, null}|{{1214697600000}}|          Putnam| {4}|           {3}|         PLAGUE SHIP|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|{{1215820800000}}|Detectives Lincol...|    {null, 0}|{{1217116800000}}|Simon & Schuster|{20}|           

*** select ***

In [10]:
dataframe.select('author').show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows



In [11]:
#Show all entries in title, author, rank, price columns
dataframe.select("author", "title", "rank", "price").show(10)

+--------------------+--------------------+----+-------------+
|              author|               title|rank|        price|
+--------------------+--------------------+----+-------------+
|       Dean R Koontz|           ODD HOURS| {1}|   {null, 27}|
|     Stephenie Meyer|            THE HOST| {2}|{25.99, null}|
|        Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|   Patricia Cornwell|           THE FRONT| {4}|{22.95, null}|
|     Chuck Palahniuk|               SNUFF| {5}|{24.95, null}|
|James Patterson a...|SUNDAYS AT TIFFANY’S| {6}|{24.99, null}|
|       John Sandford|        PHANTOM PREY| {7}|{26.95, null}|
|       Jimmy Buffett|          SWINE NOT?| {8}|{21.99, null}|
|    Elizabeth George|     CARELESS IN RED| {9}|{27.95, null}|
|      David Baldacci|     THE WHOLE TRUTH|{10}|{26.99, null}|
+--------------------+--------------------+----+-------------+
only showing top 10 rows



In [13]:
#when
dataframe.select("title",when(dataframe.title == 'ODD HOURS',1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------+
|               title|CASE WHEN (title = ODD HOURS) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------+
|           ODD HOURS|                                              1|
|            THE HOST|                                              0|
|LOVE THE ONE YOU'...|                                              0|
|           THE FRONT|                                              0|
|               SNUFF|                                              0|
|SUNDAYS AT TIFFANY’S|                                              0|
|        PHANTOM PREY|                                              0|
|          SWINE NOT?|                                              0|
|     CARELESS IN RED|                                              0|
|     THE WHOLE TRUTH|                                              0|
+--------------------+-----------------------------------------------+
only s

In [14]:
# Show rows with specified authors if in the given options
check_isin=dataframe.select("author", "title", "rank", "price")
check_isin.show(5)

+-----------------+--------------------+----+-------------+
|           author|               title|rank|        price|
+-----------------+--------------------+----+-------------+
|    Dean R Koontz|           ODD HOURS| {1}|   {null, 27}|
|  Stephenie Meyer|            THE HOST| {2}|{25.99, null}|
|     Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|Patricia Cornwell|           THE FRONT| {4}|{22.95, null}|
|  Chuck Palahniuk|               SNUFF| {5}|{24.95, null}|
+-----------------+--------------------+----+-------------+
only showing top 5 rows



In [15]:
check_isin[check_isin.author.isin("Emily Giffin","John Sandford")].show(10)

+-------------+--------------------+----+-------------+
|       author|               title|rank|        price|
+-------------+--------------------+----+-------------+
| Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|John Sandford|        PHANTOM PREY| {7}|{26.95, null}|
| Emily Giffin|LOVE THE ONE YOU'...| {4}|{24.95, null}|
|John Sandford|        PHANTOM PREY| {9}|{26.95, null}|
| Emily Giffin|LOVE THE ONE YOU'...| {4}|{24.95, null}|
|John Sandford|        PHANTOM PREY|{12}|{26.95, null}|
| Emily Giffin|LOVE THE ONE YOU'...| {5}|{24.95, null}|
| Emily Giffin|LOVE THE ONE YOU'...| {5}|{24.95, null}|
|John Sandford|        PHANTOM PREY|{20}|    {null, 0}|
| Emily Giffin|LOVE THE ONE YOU'...|{10}|{24.95, null}|
+-------------+--------------------+----+-------------+
only showing top 10 rows



<h2> Like operation </h2>

In [18]:
# Show author and title is TRUE if title has " THE " word in titles

dataframe.select('author','title',dataframe.title.like('% THE %')).show(10)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
+--------------------+--------------------+------------------+
only showing top 10 rows



<h2>“Startswith” — “ Endswith”</h2>

In [19]:
dataframe.select('author','title',dataframe.title.startswith("THE")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows



In [20]:
dataframe.select('author','title',dataframe.title.endswith("FRONT")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|endswith(title, FRONT)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                 false|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows



<h2>Substring</h2>

In [22]:
dataframe.select('author').show(5)

+-----------------+
|           author|
+-----------------+
|    Dean R Koontz|
|  Stephenie Meyer|
|     Emily Giffin|
|Patricia Cornwell|
|  Chuck Palahniuk|
+-----------------+
only showing top 5 rows



In [21]:
dataframe.select(dataframe.author.substr(1,3).alias("Author substring 1 to 3")).show(5)

+-----------------------+
|Author substring 1 to 3|
+-----------------------+
|                    Dea|
|                    Ste|
|                    Emi|
|                    Pat|
|                    Chu|
+-----------------------+
only showing top 5 rows



In [23]:
dataframe.select(dataframe.author.substr(3,6).alias("Author substring 3 to 6")).show(5)

+-----------------------+
|Author substring 3 to 6|
+-----------------------+
|                 an R K|
|                 epheni|
|                 ily Gi|
|                 tricia|
|                 uck Pa|
+-----------------------+
only showing top 5 rows



<h2>Adding columns</h2>

In [26]:
# Lit() is required while we are creating columns with exact values.
from pyspark.sql import functions as F
dataframe=dataframe.withColumn("New Column",F.lit("This is a new column"))
display(dataframe)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, New Column: string]

In [27]:
dataframe.select('New Column').show(5)

+--------------------+
|          New Column|
+--------------------+
|This is a new column|
|This is a new column|
|This is a new column|
|This is a new column|
|This is a new column|
+--------------------+
only showing top 5 rows



<h2>Updating Columns</h2>

In [28]:
# Update column 'amazon_product_url' with 'URL'
dataframe = dataframe.withColumnRenamed('amazon_product_url', 'URL')
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          New Column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, 

<h2> Removing columns </h2>

In [29]:
dataframe_remove = dataframe.drop("publisher", "published_date").show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|          New Column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  

<h2>Group by</h2>

In [30]:
dataframe.groupBy('author').count().show(10)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Kate Jacobs|    3|
|       Karen Robards|    6|
|     Gary Shteyngart|    3|
|         Lisa Genova|    7|
|James Patterson a...|   30|
|         Ruth Reichl|    3|
|    Orson Scott Card|    3|
+--------------------+-----+
only showing top 10 rows



<h2>Filter</h2>

In [39]:
# Filtering entries of title
# Only keeps records having value 'THE HOST'
dataframe.select('title','author','rank','_id','weeks_on_list').filter(dataframe["title"] == 'THE HOST').show(5)

+--------+---------------+----+--------------------+-------------+
|   title|         author|rank|                 _id|weeks_on_list|
+--------+---------------+----+--------------------+-------------+
|THE HOST|Stephenie Meyer| {2}|{5b4aa4ead3089013...|          {3}|
|THE HOST|Stephenie Meyer| {2}|{5b4aa4ead3089013...|          {4}|
|THE HOST|Stephenie Meyer| {2}|{5b4aa4ead3089013...|          {5}|
|THE HOST|Stephenie Meyer| {3}|{5b4aa4ead3089013...|          {6}|
|THE HOST|Stephenie Meyer| {3}|{5b4aa4ead3089013...|          {7}|
+--------+---------------+----+--------------------+-------------+
only showing top 5 rows



In [ ]:
# Write & Save File in .json format
dataframe.select("author", "title").write.save("Authors_Titles.json",format="json")